In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from nltk.stem.wordnet import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words("english")

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from pandas.core.frame import DataFrame

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Import Dataset

In [ ]:
train_path = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv"
comments_to_score_path = "/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv"
validation_path = "../input/jigsaw-toxic-severity-rating/validation_data.csv"
sample_path = "/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv"

In [ ]:
train = pd.read_csv(train_path)
comments_to_score = pd.read_csv(comments_to_score_path)
validation = pd.read_csv(validation_path)
sample = pd.read_csv(sample_path)

# Clean text

In [ ]:
def routine_clean(x):
    # Remove unicode characters
    x = x.encode("ascii", "ignore").decode()
    # Remove English contractions
    x = re.sub("\'\w+", '', x)
    # Remove ponctuation but not # (for C# for example)
    x = re.sub('[^\\w\\s#]', '', x)
    # Remove links
    x = re.sub(r'http*\S+', '', x)
    # Remove numbers
    x = re.sub(r'\w*\d+\w*', '', x)
    # Remove extra spaces
    x = re.sub('\s+', ' ', x)
    return x

In [ ]:
train['comment_text'] = [BeautifulSoup(text,"lxml").get_text() for text in train['comment_text']]

In [ ]:
def clean(data:DataFrame, col:str) -> DataFrame:

    data[col] = data[col].str.replace(
        'https?://\S+|www\.\S+', ' social medium ')

    data[col] = data[col].str.lower()
    data[col] = data[col].str.replace("4", "a")
    data[col] = data[col].str.replace("2", "l")
    data[col] = data[col].str.replace("5", "s")
    data[col] = data[col].str.replace("1", "i")
    data[col] = data[col].str.replace("!", "i")
    data[col] = data[col].str.replace("|", "i")
    data[col] = data[col].str.replace("0", "o")
    data[col] = data[col].str.replace("l3", "b")
    data[col] = data[col].str.replace("7", "t")
    data[col] = data[col].str.replace("7", "+")
    data[col] = data[col].str.replace("8", "ate")
    data[col] = data[col].str.replace("3", "e")
    data[col] = data[col].str.replace("9", "g")
    data[col] = data[col].str.replace("6", "g")
    data[col] = data[col].str.replace("@", "a")
    data[col] = data[col].str.replace("$", "s")
    data[col] = data[col].str.replace("#ofc", " of fuckin course ")
    data[col] = data[col].str.replace("fggt", " faggot ")
    data[col] = data[col].str.replace("your", " your ")
    data[col] = data[col].str.replace("self", " self ")
    data[col] = data[col].str.replace("cuntbag", " cunt bag ")
    data[col] = data[col].str.replace("fartchina", " fart china ")
    data[col] = data[col].str.replace("youi", " you i ")
    data[col] = data[col].str.replace("cunti", " cunt i ")
    data[col] = data[col].str.replace("sucki", " suck i ")
    data[col] = data[col].str.replace("pagedelete", " page delete ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("i'm", " i am ")
    data[col] = data[col].str.replace("offuck", " of fuck ")
    data[col] = data[col].str.replace(
        "centraliststupid", " central ist stupid ")
    data[col] = data[col].str.replace("hitleri", " hitler i ")
    data[col] = data[col].str.replace("i've", " i have ")
    data[col] = data[col].str.replace("i'll", " sick ")
    data[col] = data[col].str.replace("fuck", " fuck ")
    data[col] = data[col].str.replace("f u c k", " fuck ")
    data[col] = data[col].str.replace("shit", " shit ")
    data[col] = data[col].str.replace("bunksteve", " bunk steve ")
    data[col] = data[col].str.replace('wikipedia', ' social medium ')
    data[col] = data[col].str.replace("faggot", " faggot ")
    data[col] = data[col].str.replace("delanoy", " delanoy ")
    data[col] = data[col].str.replace("jewish", " jewish ")
    data[col] = data[col].str.replace("sexsex", " sex ")
    data[col] = data[col].str.replace("allii", " all ii ")
    data[col] = data[col].str.replace("i'd", " i had ")
    data[col] = data[col].str.replace("'s", " is ")
    data[col] = data[col].str.replace("youbollocks", " you bollocks ")
    data[col] = data[col].str.replace("dick", " dick ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("mothjer", " mother ")
    data[col] = data[col].str.replace("cuntfranks", " cunt ")
    data[col] = data[col].str.replace("ullmann", " jewish ")
    data[col] = data[col].str.replace("mr.", " mister ")
    data[col] = data[col].str.replace("aidsaids", " aids ")
    data[col] = data[col].str.replace("njgw", " nigger ")
    data[col] = data[col].str.replace("wiki", " social medium ")
    data[col] = data[col].str.replace("administrator", " admin ")
    data[col] = data[col].str.replace("gamaliel", " jewish ")
    data[col] = data[col].str.replace("rvv", " vanadalism ")
    data[col] = data[col].str.replace("admins", " admin ")
    data[col] = data[col].str.replace("pensnsnniensnsn", " penis ")
    data[col] = data[col].str.replace("pneis", " penis ")
    data[col] = data[col].str.replace("pennnis", " penis ")
    data[col] = data[col].str.replace("pov.", " point of view ")
    data[col] = data[col].str.replace("vandalising", " vandalism ")
    data[col] = data[col].str.replace("cock", " dick ")
    data[col] = data[col].str.replace("asshole", " asshole ")
    data[col] = data[col].str.replace("youi", " you ")
    data[col] = data[col].str.replace("afd", " all fucking day ")
    data[col] = data[col].str.replace("sockpuppets", " sockpuppetry ")
    data[col] = data[col].str.replace("iiprick", " iprick ")
    data[col] = data[col].str.replace("penisi", " penis ")
    data[col] = data[col].str.replace("warrior", " warrior ")
    data[col] = data[col].str.replace("loil", " laughing out insanely loud ")
    data[col] = data[col].str.replace("vandalise", " vanadalism ")
    data[col] = data[col].str.replace("helli", " helli ")
    data[col] = data[col].str.replace("lunchablesi", " lunchablesi ")
    data[col] = data[col].str.replace("special", " special ")
    data[col] = data[col].str.replace("ilol", " i lol ")
    data[col] = data[col].str.replace(r'\b[uU]\b', 'you')
    data[col] = data[col].str.replace(r"what's", "what is ")
    data[col] = data[col].str.replace(r"\'s", " is ")
    data[col] = data[col].str.replace(r"\'ve", " have ")
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ")
    data[col] = data[col].str.replace(r"\'d", " would ")
    data[col] = data[col].str.replace(r"\'ll", " will ")
    data[col] = data[col].str.replace(r"\'scuse", " excuse ")
    # will remove more than one whitespace character
    data[col] = data[col].str.replace('\s+', ' ')
#     text = re.sub(r'\b([^\W\d_]+)(\s+\1)+\b', r'\1', re.sub(r'\W+', ' ', text).strip(), flags=re.I)  # remove repeating words coming immediately one after another
    # 2 or more characters are replaced by 2 characters
    data[col] = data[col].str.replace(r'(.)\1+', r'\1\1')
#     text = re.sub(r'((\b\w+\b.{1,2}\w+\b)+).+\1', r'\1', text, flags = re.I)
    data[col] = data[col].str.replace(
        "[:|♣|'|§|♠|*|/|?|=|%|&|-|#|•|~|^|>|<|►|_]", '')

    data[col] = data[col].str.replace(r"what's", "what is ")
    data[col] = data[col].str.replace(r"\'ve", " have ")
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ")
    data[col] = data[col].str.replace(r"\'d", " would ")
    data[col] = data[col].str.replace(r"\'ll", " will ")
    data[col] = data[col].str.replace(r"\'scuse", " excuse ")
    data[col] = data[col].str.replace(r"\'s", " ")

    # Clean some punctutations
    data[col] = data[col].str.replace('\n', ' \n ')
    data[col] = data[col].str.replace(
        r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)', r'\1 \2 \3')
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}', r'\1\1\1')
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\']+)', r' \1 ')
    # patterns with repeating characters
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b', r'\1\1')
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B', r'\1\1\1')
    data[col] = data[col].str.replace(r'[ ]{2,}', ' ').str.strip()
    data[col] = data[col].str.replace(r'[ ]{2,}', ' ').str.strip()
    data[col] = data[col].apply(lambda x: ' '.join(
        [word for word in x.split() if word not in (stop_words)]))

    return data

In [ ]:
df_train = clean(train, 'comment_text')

In [ ]:
df_train['comment_text'] = df_train['comment_text'].str.lower()

# Universal Sentence Encoder

In [ ]:
import tensorflow_hub as hub

path_encoder = "/kaggle/input/universalsentenceencoder4/"
encoder = hub.load(path_encoder)

In [ ]:
import tensorflow as tf

train_1 = df_train['comment_text'][0:10000]
X_train_encoder_use = encoder(train_1)
X_train_encoder_use = X_train_encoder_use['outputs']

for i in range (1,16):
    j = i + 1
    i0 = i*10000
    j0 = j*10000
    split = df_train['comment_text'][i0:j0]
    split_encoder = encoder(split)
    X_train_encoder_use = tf.concat([X_train_encoder_use, split_encoder['outputs']], 0)

In [ ]:
X_train_encoder_use = X_train_encoder_use.numpy()

In [ ]:
validation['less_toxic'] = [BeautifulSoup(text,"lxml").get_text() for text in validation['less_toxic']]
validation['less_toxic'] = [routine_clean(text) for text in validation['less_toxic']]
df_validation = clean(validation, 'less_toxic')
df_validation['less_toxic'] = df_validation['less_toxic'].str.lower()


validation['more_toxic'] = [BeautifulSoup(text,"lxml").get_text() for text in validation['more_toxic']]
validation['more_toxic'] = [routine_clean(text) for text in validation['more_toxic']]
df_validation = clean(validation, 'more_toxic')
df_validation['more_toxic'] = df_validation['more_toxic'].str.lower()

train_1 = df_validation['less_toxic'][0:10000]
validation_less_encoder_use = encoder(train_1)
validation_less_encoder_use = validation_less_encoder_use['outputs']

train_1 = df_validation['more_toxic'][0:10000]
validation_more_encoder_use = encoder(train_1)
validation_more_encoder_use = validation_more_encoder_use['outputs']

for i in range (1,4):
    j = i + 1
    i0 = i*10000
    j0 = j*10000
    split = df_validation['less_toxic'][i0:j0]
    split_encoder = encoder(split)
    validation_less_encoder_use = tf.concat([validation_less_encoder_use, split_encoder['outputs']], 0)   
    split = df_validation['more_toxic'][i0:j0]
    split_encoder = encoder(split)
    validation_more_encoder_use = tf.concat([validation_more_encoder_use, split_encoder['outputs']], 0)

In [ ]:
validation_less_encoder_use = validation_less_encoder_use.numpy()
validation_more_encoder_use = validation_more_encoder_use.numpy()

# Sentence Transformer

Encode localy and export as Dataset with All mini l12 model

In [ ]:
X_train_encoder_sbert_path = "../input/data-encoder-all-mini-l12-v2/Data_encoder_all_Mini.csv"
validation_less_encoder_sbert_path = "../input/data-encoder-all-mini-l12-v2/Data_less_encoder_all_Mini.csv"
validation_more_encoder_sbert_path = "../input/data-encoder-all-mini-l12-v2/Data_more_encoder_all_Mini.csv"
comments_encoder_sbert_path = "../input/data-encoder-all-mini-l12-v2/Data_comments_encoder_all_Mini.csv"

X_train_encoder_sbert = pd.read_csv(X_train_encoder_sbert_path)
validation_less_encoder_sbert = pd.read_csv(validation_less_encoder_sbert_path)
validation_more_encoder_sbert = pd.read_csv(validation_more_encoder_sbert_path)
comments_encoder_sbert = pd.read_csv(comments_encoder_sbert_path)

# Concatenate

In [ ]:
X_train_encoder_use = pd.DataFrame(X_train_encoder_use)
X_train_sbert_use = pd.concat((X_train_encoder_sbert, X_train_encoder_use),axis=1)
X_train_sbert_use = X_train_sbert_use.T.reset_index(drop=True).T

In [ ]:
validation_less_encoder_use = pd.DataFrame(validation_less_encoder_use)
validation_more_encoder_use = pd.DataFrame(validation_more_encoder_use)

In [ ]:
X_less_toxic = pd.concat((validation_less_encoder_sbert, validation_less_encoder_use),axis=1)
X_more_toxic = pd.concat((validation_more_encoder_sbert, validation_more_encoder_use),axis=1)

# Comments to score

In [ ]:
comments_to_score['text'] = [BeautifulSoup(text,"lxml").get_text() for text in comments_to_score['text']]
comments_to_score['text'] = [routine_clean(text) for text in comments_to_score['text']]
df_comment = clean(comments_to_score, 'text')
df_comment['text'] = df_comment['text'].str.lower()


In [ ]:
comment_toxic_use = encoder(df_comment['text'])
comment_toxic_use = comment_toxic_use['outputs']
comment_toxic_use = comment_toxic_use.numpy()
comment_toxic_use = pd.DataFrame(comment_toxic_use)
X_comment_toxic = pd.concat((comments_encoder_sbert, comment_toxic_use),axis=1)

# Train

In [ ]:
df_train['sum'] = 0.32*train['toxic'] + 1.5*train['severe_toxic'] + 0.16*train['obscene'] + 1.5*train['threat'] + 0.64*train['insult'] + 1.5*train['identity_hate']
label = df_train['sum']

X_train_sbert_use, X_val_sbert_use, y_train, y_val = train_test_split(X_train_sbert_use, label, test_size=0.1, random_state=42)

train_data = lgb.Dataset(X_train_sbert_use, label=y_train)
val_data = lgb.Dataset(X_val_sbert_use, label=y_val)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 425,
    'max_depth': 25,
    "feature_fraction": 0.8,
    'subsample_freq': 1,
    "bagging_fraction": 0.98,
    'min_data_in_leaf': 20,
    'learning_rate': 0.05,
    "boosting": "gbdt",
    "lambda_l1": 0.2,
    "lambda_l2": 10,
    "verbosity": -1,
    "random_state": 42
}


In [ ]:
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[val_data])

In [ ]:
# Validation Score

less_toxic_pred_sbert_use = bst.predict(X_less_toxic)
more_toxic_pred_sbert_use = bst.predict(X_more_toxic)
(less_toxic_pred_sbert_use < more_toxic_pred_sbert_use).mean()

In [ ]:
# make predictions
pred_comment_1 = bst.predict(X_comment_toxic)

In [ ]:
result_1 = pred_comment_1

In [ ]:
sample['score'] = result_1

In [ ]:
sample.to_csv('./submission.csv', index=False)